In [42]:
from pymongo import MongoClient

In [43]:
client = MongoClient()

In [44]:
db = client["ided"]
course_col = db["courses"]

In [45]:
# Get the course to id mappings
course_to_id = {}
for doc in course_col.find():
    course_to_id[doc["course_id"]] = doc["_id"]

In [46]:
# Get the wildcard course to id mappings
wildcard_col = db["wildcardcourses"]
wild_to_id = {}
for doc in wildcard_col.find():
    wild_to_id[doc["course_id"]] = doc["_id"]

In [47]:
# This function replaces the course and wildcard course names in courses collection with ObjectIds
def id_prereq_elem(elem):
    # elem is an array of prerequisite strings
    ided_elem = []
    for prereq_str in elem:
        if prereq_str not in course_to_id:
            print("Prerequisite not in course_to_id: ", prereq_str)
            continue
        ided_elem.append(course_to_id[prereq_str])
    return ided_elem

# Replace the strings in prerequisite arrays with ids
for doc in course_col.find():
    if "prerequisite" not in doc:
        print("prerequisite field not in: ", doc["course_id"])
        continue
    prereqs = doc["prerequisite"]
    ided_prereqs = [id_prereq_elem(elem) for elem in prereqs]
    course_col.update_one(
        { '_id': doc["_id"] },
        { "$set": { "prerequisite": ided_prereqs } }
    )

prerequisite field not in:  COMPSCI 119
Prerequisite not in course_to_id:  CICS 160
prerequisite field not in:  COMPSCI 220
Prerequisite not in course_to_id:  CICS 160
prerequisite field not in:  COMPSCI 311
prerequisite field not in:  COMPSCI 333
prerequisite field not in:  COMPSCI 335
prerequisite field not in:  COMPSCI 360
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  STAT 515
prerequisite field not in:  COMPSCI 403
prerequisite field not in:  COMPSCI 474
prerequisite field not in:  COMPSCI 501
prerequisite field not in:  COMPSCI 503
Prerequisite not in course_to_id:  STAT 515
prerequisite field not in:  COMPSCI 560
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  STAT 515
prerequisite field not in:  COMPSCI 578
Prerequisite not in course_to_id:  STAT 515
Prerequisite not in course_to_id:  C

In [48]:
# Replace the strings in associated_courses with ids
for doc in wildcard_col.find():
    courses = doc["associated_courses"]
    ided_courses = []
    for course in courses:
        if course not in course_to_id:
            print("Course not in course_to_id: ", course)
            continue
        ided_courses.append(course_to_id[course])
    wildcard_col.update_one(
        { "_id": doc["_id"] },
        { "$set": { "associated_courses": ided_courses } }
    )

Course not in course_to_id:  ECE 353
Course not in course_to_id:  LINGUIST 401
Course not in course_to_id:  CHEM 121
Course not in course_to_id:  GEOL 10


In [49]:
req_col = db["requirements"]

In [50]:
for doc in req_col.find():
    all_courses = doc["courses"]
    courses, wildcard_courses = [], []
    for course_arr in all_courses:
        courses_arr, wildcard_courses_arr = [], []
        for course in course_arr:
            if course not in course_to_id and course not in wild_to_id:
                print("Course is neither concrete nor wildcard: ", course)
                continue
            if course in wild_to_id:
                wildcard_courses_arr.append(wild_to_id[course])
            else:       # course in course_to_id
                courses_arr.append(course_to_id[course])
        if courses_arr:
            courses.append(courses_arr)
        if wildcard_courses_arr:
            wildcard_courses.append(wildcard_courses_arr)
    if courses and wildcard_courses:
        print("This req has both wildcard and normal requirements: ", doc["requirement"])
        continue
    if not courses and not wildcard_courses:
        print("This req has no requirements: ", doc["requirement"])

    # Update the fields
    req_col.update_one(
        { "_id": doc["_id"] },
        {
            "$rename": { "requirement": "description" },
            "$set": {
                "courses": courses,
                "wildcardCourses": wildcard_courses
            }
        }
    )

Course is neither concrete nor wildcard:  STAT 515


In [51]:
major_col = db["majors"]
cs_reqs = [doc["_id"] for doc in req_col.find()]
major_col.insert_one({
    "name": "BS CS",
    "requirements": cs_reqs
})